<h1> Exploracion de los Datos (EDA) </h1>

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_parquet('../data/processed/online_retail_II_cleaned.parquet')

In [22]:
# Confiuracion inicial para graficos
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## Verificacion Inicial de los Datos

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400916 entries, 0 to 400915
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      400916 non-null  string        
 1   stockcode    400916 non-null  string        
 2   description  400916 non-null  string        
 3   quantity     400916 non-null  int16         
 4   invoicedate  400916 non-null  datetime64[ns]
 5   price        400916 non-null  float32       
 6   customer_id  400916 non-null  Int32         
 7   country      400916 non-null  string        
dtypes: Int32(1), datetime64[ns](1), float32(1), int16(1), string(4)
memory usage: 19.5 MB


In [24]:
print(f"Rango de Fechas: {df['invoicedate'].dt.date.min()} a {df['invoicedate'].dt.date.max()}")
print(f"Total Clientes Únicos: {df['customer_id'].nunique()}")

Rango de Fechas: 2009-12-01 a 2010-12-09
Total Clientes Únicos: 4312


Se valida que el dataset sea temporalmente suficiente y estadísticamente adecuado para estimar el CLTV en un horizonte de 6 meses. Ademas se verifica el tamaño de clientes unicos en todo el dataset y para el periodo de entrenamiento.

## Creacion de Variable Monetaria

In [25]:
df['total_spend'] = df['quantity'] * df['price']
df.head()

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,total_spend
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.399994
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.000000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.000000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.799995
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.000000


Calcular el monto monetario de cada transacción, que será la base para estimar el valor económico del cliente en el modelo de CLTV a 6 meses.

- Permite medir cuánto gasta cada cliente en cada compra.
- Facilita la agregación posterior a nivel cliente (gasto promedio, gasto total).